In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from seaborn import set_style
import kagglehub
set_style("whitegrid")

In [2]:
path = kagglehub.dataset_download("ehallmar/daily-historical-stock-prices-1970-2018")

print("Path to dataset files:", path)

Path to dataset files: /home/hava/.cache/kagglehub/datasets/ehallmar/daily-historical-stock-prices-1970-2018/versions/1


In [3]:
prices_file = path + '/historical_stock_prices.csv'
labels_file = path + '/historical_stocks.csv'
df_prices = pd.read_csv(prices_file)
df_labels = pd.read_csv(labels_file)
df_prices

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14
...,...,...,...,...,...,...,...,...
20973884,NZF,14.60,14.59,14.590000,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.580000,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.590000,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.570000,14.57,14.64,135600,2018-08-23


In [4]:
sector = df_labels[df_labels['sector']=='ENERGY'][['ticker','name']]
sector['ticker']=sector['ticker'].astype(str)
print(sector['ticker'])
df_sector = df_prices[df_prices['ticker'].isin(sector['ticker'])].reset_index()
df_sector

20      AXAS
112     AHGP
114     ARLP
152     AETI
202     AREX
        ... 
6413     WRD
6423     INT
6428     WPX
6429    WPXP
6445     YPF
Name: ticker, Length: 286, dtype: object


,index,ticker,open,close,adj_close,low,high,volume,date
0,18698,RRC,2.83333,2.83333,2.486157,2.83333,2.83333,400,1992-12-28
1,18710,RRC,2.83333,2.83333,2.486157,2.66667,2.83333,5400,1992-12-29
2,18711,RRC,2.66667,2.66667,2.339919,2.58333,2.66667,17200,1992-12-30
3,18712,RRC,2.58333,2.83333,2.486157,2.58333,2.83333,1500,1992-12-31
4,18721,RRC,2.58333,2.58333,2.266791,2.58333,2.58333,100,1993-01-04
...,...,...,...,...,...,...,...,...,...
1101553,20788580,GE,12.27000,12.30000,12.300000,12.14000,12.39000,37756500,2018-08-20
1101554,20788582,GE,12.34000,12.63000,12.630000,12.33000,12.69000,57947600,2018-08-21
1101555,20788584,GE,12.61000,12.47000,12.470000,12.42000,12.62000,43446200,2018-08-22
1101556,20788585,GE,12.41000,12.54000,12.540000,12.39000,12.56000,38054600,2018-08-23


In [5]:
df_sector['date'] = pd.to_datetime(df_sector['date'])
df_sector['ticker'] = (df_sector['ticker']).astype(str)
dst_dates_df = pd.read_csv('DaylightSavingsTimeChangeDates_1971-2024.csv')
dst_dates_spring = pd.to_datetime(dst_dates_df.Date+" "+dst_dates_df.Year.astype(str))
dst_dates_fall = pd.to_datetime(dst_dates_df['Date.1']+" "+dst_dates_df.Year.astype(str))
dst_dates = pd.concat([dst_dates_fall,dst_dates_spring])
dst_mon = dst_dates+pd.DateOffset(1)
dst_fri = dst_dates+pd.DateOffset(-2)
dst_fall_mon = dst_dates_fall+pd.DateOffset(1)
dst_fall_fri = dst_dates_fall+pd.DateOffset(-2)
dst_spring_mon = dst_dates_spring+pd.DateOffset(1)
dst_spring_fri = dst_dates_spring+pd.DateOffset(-2)

In [6]:
sector_dst_mon_fall = df_sector[df_sector.date.isin(dst_fall_mon)]
sector_dst_mon_fall = sector_dst_mon_fall.assign(Season='Fall')
sector_dst_mon_spring = df_sector[df_sector.date.isin(dst_spring_mon)]
sector_dst_mon_spring = sector_dst_mon_spring.assign(Season='Spring')
sector_dst_mon = pd.concat([sector_dst_mon_fall,sector_dst_mon_spring])
sector_dst_fri = df_sector[df_sector.date.isin(dst_fri)]

for tkr in sector['ticker'].astype(str):
    mon_tkr = sector_dst_mon[sector_dst_mon.ticker.astype(str)==tkr]
    fri_tkr = sector_dst_fri[sector_dst_fri.ticker.astype(str)==tkr]
    sector_dst_mon = sector_dst_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    sector_dst_fri = sector_dst_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
    #finance_dst_mon = finance_dst_mon.drop(finance_dst_mon[(finance_dst_mon.ticker.astype(str)==tkr  finance_dst_mon['date'].isin(finance_dst_fri[finance_dst_fri.ticker.astype(str)==tkr].date+pd.DateOffset(3))==False)].index)
    #finance_dst_fri = finance_dst_fri.drop(finance_dst_fri[(finance_dst_fri['ticker'==tkr] & finance_dst_fri['date'].isin(finance_dst_mon['ticker'==tkr].date+pd.DateOffset(-3))==False)].index)
print(len(sector_dst_mon))
print(len(sector_dst_fri))

8263
8263


In [7]:
sector_dst_fri = sector_dst_fri.reset_index()
sector_dst_mon = sector_dst_mon.reset_index()

In [8]:
sector_dst = pd.DataFrame()
sector_dst['ticker'] = sector_dst_mon['ticker']
sector_dst['mon_date'] = sector_dst_mon['date']
sector_dst['mon_open'] = sector_dst_mon['open']
sector_dst['mon_close'] = sector_dst_mon['close']
sector_dst['mon_adj_close'] = sector_dst_mon['adj_close']
sector_dst['mon_low'] = sector_dst_mon['low']
sector_dst['mon_high'] = sector_dst_mon['high']
sector_dst['mon_volume'] = sector_dst_mon['volume']
sector_dst['fri_date'] = sector_dst_fri['date']
sector_dst['fri_open'] = sector_dst_fri['open']
sector_dst['fri_close'] = sector_dst_fri['close']
sector_dst['fri_adj_close'] = sector_dst_fri['adj_close']
sector_dst['fri_low'] = sector_dst_fri['low']
sector_dst['fri_high'] = sector_dst_fri['high']
sector_dst['fri_volume'] = sector_dst_fri['volume']
sector_dst['Season'] = sector_dst_mon['Season']
sector_dst

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume,Season
0,RRC,1993-11-01,6.000000,6.000000,5.264811,5.833330,6.000000,17700,1993-04-02,4.166670,4.000000,3.509874,3.833330,4.250000,54000,Fall
1,RRC,1994-10-31,5.083330,5.166670,4.533590,5.000000,5.166670,10300,1993-10-29,5.916670,6.000000,5.264811,5.750000,6.000000,42600,Fall
2,RRC,1995-10-30,5.166670,5.208330,4.570147,5.166670,5.333330,285700,1994-10-28,5.083330,5.083330,4.460460,4.916670,5.166670,32100,Fall
3,RRC,1996-10-28,10.500000,10.666670,9.421787,10.416670,10.666670,64500,1995-03-31,4.666670,4.666670,4.094858,4.583330,4.750000,65400,Fall
4,RRC,1997-10-27,12.333330,11.416670,10.163116,11.333330,12.458330,183100,1995-10-27,5.166670,5.333330,4.679830,5.083330,5.333330,53800,Fall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8258,GE,2014-03-10,26.120001,26.040001,22.276007,25.840000,26.129999,22263500,2016-03-11,30.139999,30.340000,27.779388,30.139999,30.580000,32159600,Spring
8259,GE,2015-03-09,25.520000,25.639999,22.718422,25.490000,25.700001,31591100,2016-11-04,28.219999,28.440001,26.441784,28.219999,28.809999,35864900,Spring
8260,GE,2016-03-14,30.219999,30.270000,27.715298,30.080000,30.370001,22160300,2017-03-10,29.840000,30.280001,28.590612,29.629999,30.430000,59211900,Spring
8261,GE,2017-03-13,30.230000,29.860001,28.194048,29.790001,30.260000,30187800,2017-11-03,19.920000,20.139999,19.530336,19.860001,20.330000,67861000,Spring


In [9]:
sector_mon = df_sector[df_sector.date.dt.day_name()=='Monday']
sector_mon = sector_mon.drop(sector_mon[sector_mon.date.isin(dst_mon)].index)
sector_fri = df_sector[df_sector.date.dt.day_name()=='Friday']
sector_fri = sector_fri.drop(sector_fri[sector_fri.date.isin(dst_fri)].index)
for tkr in sector['ticker'].astype(str):
    mon_tkr = sector_mon[sector_mon.ticker.astype(str)==tkr]
    fri_tkr = sector_fri[sector_fri.ticker.astype(str)==tkr]
    sector_mon = sector_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    sector_fri = sector_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
print(len(sector_mon))
print(len(sector_fri))

188560
188560


In [10]:
sector_mon = sector_mon.reset_index()
sector_fri = sector_fri.reset_index()

In [11]:
sector_wkd = pd.DataFrame()
sector_wkd['ticker'] = sector_mon['ticker']
sector_wkd['mon_date'] = sector_mon['date']
sector_wkd['mon_open'] = sector_mon['open']
sector_wkd['mon_close'] = sector_mon['close']
sector_wkd['mon_adj_close'] = sector_mon['adj_close']
sector_wkd['mon_low'] = sector_mon['low']
sector_wkd['mon_high'] = sector_mon['high']
sector_wkd['mon_volume'] = sector_mon['volume']
sector_wkd['fri_date'] = sector_fri['date']
sector_wkd['fri_open'] = sector_fri['open']
sector_wkd['fri_close'] = sector_fri['close']
sector_wkd['fri_adj_close'] = sector_fri['adj_close']
sector_wkd['fri_low'] = sector_fri['low']
sector_wkd['fri_high'] = sector_fri['high']
sector_wkd['fri_volume'] = sector_fri['volume']
sector_wkd

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume
0,RRC,1993-01-11,2.66667,2.58333,2.266791,2.58333,2.66667,2800,1993-01-08,2.66667,2.58333,2.266791,2.58333,2.66667,15000
1,RRC,1993-01-18,2.66667,2.70833,2.376474,2.66667,2.83333,17800,1993-01-15,2.66667,2.66667,2.339919,2.66667,2.70833,3900
2,RRC,1993-01-25,2.75000,2.83333,2.486157,2.75000,2.83333,28200,1993-01-22,2.83333,2.79167,2.449601,2.79167,2.83333,17400
3,RRC,1993-02-01,2.83333,2.75000,2.413038,2.75000,2.83333,4300,1993-01-29,2.83333,2.83333,2.486157,2.83333,2.83333,3000
4,RRC,1993-02-08,2.75000,2.75000,2.413038,2.75000,2.75000,1800,1993-02-05,2.75000,2.83333,2.486157,2.66667,2.83333,70800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188555,GE,2018-07-23,13.02000,12.99000,12.990000,12.90000,13.12000,90332800,2018-07-20,13.72000,13.12000,13.120000,12.98000,13.78000,171245400
188556,GE,2018-07-30,13.04000,13.16000,13.160000,13.01000,13.21000,39565000,2018-07-27,13.12000,13.06000,13.060000,12.98000,13.18000,39330600
188557,GE,2018-08-06,13.12000,13.10000,13.100000,13.06000,13.18000,32489300,2018-08-03,13.15000,13.14000,13.140000,13.10000,13.25000,35245100
188558,GE,2018-08-13,12.72000,12.45000,12.450000,12.39000,12.73000,67070800,2018-08-10,12.91000,12.77000,12.770000,12.71000,12.91000,48452300


In [12]:
sector_dst.to_csv('DST_Energy_Stocks.csv')
sector_wkd.to_csv('Weekends_Energy_Stocks.csv')